In [1]:
from nltk.tokenize import sent_tokenize
import pandas as pd
import spacy
from spacy.lang.en import English
import numpy as np
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

C:\Users\gargk\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\gargk\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\gargk\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
df = pd.read_csv("news_final_project_with_topics.csv")

df.head()

,date,language,title,text,text_clean,topic
0,2022-01-06,english,"No deal, no school: Chicago cancels classes fo...",CHICAGO (AP) — Chicago school leaders canceled...,CHICAGO AP Chicago school leaders canceled cl...,1
1,2022-01-06,english,"No deal, no school: Chicago cancels classes fo...",CHICAGO (AP) — Chicago school leaders canceled...,CHICAGO AP Chicago school leaders canceled cl...,1
2,2022-01-06,english,Watch ‘Chicago P.D.’ Preview Wednesday,Chicago PD 9×11 “Lies” Season 9 Episode 11 Pro...,Chicago PD 911 Lies Season 9 Episode 11 Promo ...,3
3,2022-01-06,english,Trump’s Solution for Chicago Public Schools? K...,Wise of Foolish?\nThe Chicago Tribune article ...,Wise of FoolishThe Chicago Tribune article tit...,3
4,2022-01-06,english,"Family, friends, colleagues gathering today at...",The neighbors near Mashawn Plummer’s Portage P...,The neighbors near Mashawn Plummers Portage Pa...,3


## Perprocessing

Removing website related text

In [92]:
df['text_clean'] = df['text_clean'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)


In [96]:
df.loc[df['topic'] == 6, 'text_clean'][546]

'LAWRENCEVILLE, Ill.  An Illinois man has been sentenced to 30 years in prison for solicitation of child pornography and aggravated criminal sexual abuse , Attorney General Kwame Raoul announced Tuesday.An investigation into Glen. E. Howder, Jr., 59, of Lawrenceville, began in September 2017 after a minor victim came forward to law enforcement about the ongoing abuse.DOWNLOAD THE FOX 32 CHICAGO APP FOR BREAKING NEWS ALERTSOfficers searched Howders home and seized various electronic devices, officials said. After examining the devices, charges were filed against Howder.Individuals who exploit and abuse children need to be held accountable for their horrific actions, Raoul said in a statement.AdvertisementHowder faces additional pending charges in Edwards County for possession of child pornography.'

### Converting text to sentences

In [101]:


nlp = English()  # just the language with no model
sentencizer = nlp.create_pipe("sentencizer")
nlp.add_pipe('sentencizer')


%time
# # SpaCy with sentencizer
df.loc[df['topic'] == 6, 'text_clean'].apply(lambda x: [sent.text for sent in nlp(x).sents])

CPU times: total: 0 ns
Wall time: 0 ns


159       [Chicago PD S09E10 720p WEB H264DEXTEROUSFeel free to post any comments about Chicago PD S09E10 720p WEB H264DEXTEROUS, including links to downloads, samples, screenshots, information, or any other relevant information., Please note that anything you download from here, you do so at your own risk., Remember if you see , this means they are trusted commentors Chicago., PD.S09E10.480p.x264mSD  Chicago PD S09E10 720p WEB H264DEXTEROUS  Chicago PD S09E10 1080p WEB H264DEXTEROUS  Chicago., PD.S09E10.720p., HEVC.x265MeGusta   2.75 GB  1.47 GB Pd S09E10 720p Web H264 dexterous mkvsize: 1,47 GBDropapk: Pd S09E10 Web H264 RBB mp4size: 318,40 MBDropapk: Pd S09E10 Xvid AFG avisize: 396,08 MBDropapk: Pd S09E10 Web X264 TGX mkvsize: 289,25 MBDropapk: Pd S09E10 480p X264 mSD mkvsize: 303,57 MBDropapk: Pd S09E10 720p Hevc X265 MeGusta mkvsize: 360,99 MBDropapk: : 303.57 MB : 360.99 MB : 1.47 GB : 2...
546                                                                                       

In [105]:
df1 = df[['date', 'text_clean', 'topic']]

del df

### Loading spacy en_core_web_sm model to convert text to sentences

In [114]:
nlp = spacy.load("en_core_web_sm")
df1["text_clean"] = df1["text_clean"].astype(str).apply(lambda x: [sent.text for sent in nlp(x).sents])
df1 = df1.explode("text_clean", ignore_index=True)

In [117]:
df1.head()

,date,text_clean,topic
0,2022-01-06,CHICAGO AP Chicago school leaders canceled classes Thursday for a second consecutive day after failing to reach agreement with the teachers union over COVID19 safety protocols in the nations thirdlargest school district.,1
1,2022-01-06,The Chicago Teachers Union sought to revert to remote instruction during the latest surge of infections and while both sides hammer out a deal.,1
2,2022-01-06,But Chicago Public Schools leaders have said remote learning didnt work and schools can safely remain open with protocols in place.,1
3,2022-01-06,"The move to cancel classes and activities Thursday affects roughly 350,000 students and came after closeddoor negotiations Wednesday failed to produce a deal.",1
4,2022-01-06,The issues include metrics for closing schools.,1


In [120]:
df1.shape

(4531961, 3)

Dropping na and texts less than length 30 as we cannot get sentiments for those

In [182]:
df1['text_clean'].dropna(inplace=True)

df1 = df1[~(df1.text_clean.str.len() <=30)]

## Loading yelp trained model

In [3]:
%time clf_large = load("/home/jupyter/data/yelp/yelp_model/logreg.joblib")

CPU times: total: 31.5 s
Wall time: 31.7 s


Using yelp model to predict sentiment on text

In [206]:

def apply_sentiments(content):
    try:
        y_pred = clf_large.predict([content])
        y_pred_class = np.where(y_pred < 1, 'Negative', 'Positive')
        y_pred_prob = clf_large.predict_proba([content])
        return y_pred_class[0], y_pred_prob[0][0], y_pred_prob[0][1]
    except:
        return np.nan
    
df1['sentiments'] = df1.progress_apply(lambda x: apply_sentiments(x["text_clean"]),axis=1)

C:\Users\gargk\AppData\Local\Temp\ipykernel_37484\392814080.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


0it [00:00, ?it/s]

  0%|          | 0/4054659 [00:00<?, ?it/s]

C:\Users\gargk\AppData\Local\Temp\ipykernel_37484\392814080.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['sentiments'] = df1.progress_apply(lambda x: apply_sentiments(x["text_clean"]),axis=1)


In [207]:
df1.head()

,date,text_clean,topic,sentiments
0,2022-01-06,CHICAGO AP Chicago school leaders canceled classes Thursday for a second consecutive day after failing to reach agreement with the teachers union over COVID19 safety protocols in the nations thirdlargest school district.,1,"(Negative, 0.8961695106614529, 0.10383048933854715)"
1,2022-01-06,The Chicago Teachers Union sought to revert to remote instruction during the latest surge of infections and while both sides hammer out a deal.,1,"(Positive, 0.4675166632374851, 0.5324833367625149)"
2,2022-01-06,But Chicago Public Schools leaders have said remote learning didnt work and schools can safely remain open with protocols in place.,1,"(Negative, 0.7853307303079926, 0.21466926969200734)"
3,2022-01-06,"The move to cancel classes and activities Thursday affects roughly 350,000 students and came after closeddoor negotiations Wednesday failed to produce a deal.",1,"(Negative, 0.9643950816841969, 0.03560491831580317)"
4,2022-01-06,The issues include metrics for closing schools.,1,"(Negative, 0.6093631112708314, 0.3906368887291686)"


Dropping missing sentiment rows and saving the dataframe

In [228]:
df1 = df1[~(df1.sentiments.isna())]
df1.reset_index(inplace = True)
df1[['sentiment', 'proba_neg', 'proba_pos']] = pd.DataFrame(df1['sentiments'].tolist())
df1.drop(['index','sentiments'], axis = 1, inplace = True)

In [239]:
df1.head()

,date,text_clean,topic,sentiment,proba_neg,proba_pos
0,2022-01-06,CHICAGO AP Chicago school leaders canceled classes Thursday for a second consecutive day after failing to reach agreement with the teachers union over COVID19 safety protocols in the nations thirdlargest school district.,1,Negative,0.896170,0.103830
1,2022-01-06,The Chicago Teachers Union sought to revert to remote instruction during the latest surge of infections and while both sides hammer out a deal.,1,Positive,0.467517,0.532483
2,2022-01-06,But Chicago Public Schools leaders have said remote learning didnt work and schools can safely remain open with protocols in place.,1,Negative,0.785331,0.214669
3,2022-01-06,"The move to cancel classes and activities Thursday affects roughly 350,000 students and came after closeddoor negotiations Wednesday failed to produce a deal.",1,Negative,0.964395,0.035605
4,2022-01-06,The issues include metrics for closing schools.,1,Negative,0.609363,0.390637


In [240]:
df1.to_csv("text_tokenized.csv")

## Visualize results

In [63]:
text_clean = df.loc[df['topic'] == 5, 'text_clean'].str.cat(sep='. ')
content = sent_tokenize(text_clean)

In [64]:
y_pred = clf_large.predict(content)
y_pred_class = np.where(y_pred < 1, 'Negative', 'Positive')
y_pred_prob = clf_large.predict_proba(content)

print(f'Text: {content[0]} \
    \nSentiment: {y_pred_class[0]} \
    \nProbability Negative: = {y_pred_prob[0][0]}\
    \nProbability Positive = {y_pred_prob[0][1]}\
    \n\n')

Text: Not known Facts About near me roofing contractor Chicago ILTerminate or modify appointment at any time By clicking Confirm Appointment, you affirm youve examine and agree to the HomeAdvisor Conditions  Conditions, and also you concur and authorize HomeAdvisor and its affiliate marketers, and their networks of Service Professionals, to provide advertising phone calls or texts employing automated technological innovation for the amount you offered above about your job together with other property companies provides.     
Sentiment: Negative     
Probability Negative: = 0.9932701468071302    
Probability Positive = 0.0067298531928697445    


